In [ ]:
!pip install ultralytics opencv-python-headless rarfile tqdm torch torchvision requests
!apt-get install unrar  # Install unrar for extracting .rar files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define paths
data_dir = "/content/UCF101"
video_dir = os.path.join(data_dir, "videos")
annotation_dir = os.path.join(data_dir, "annotations")
yolo_data_dir = "/content/yolo_dataset"

# Create directories
os.makedirs(data_dir, exist_ok=True)
os.makedirs(video_dir, exist_ok=True)
os.makedirs(annotation_dir, exist_ok=True)
os.makedirs(yolo_data_dir, exist_ok=True)

print("Directories created successfully!")

Directories created successfully!


In [ ]:
import requests
from tqdm import tqdm
import os
from google.colab import files

# URLs for UCF101 dataset and annotations
video_url = "https://www.crcv.ucf.edu/data/UCF101/UCF101.rar"
train_test_split_url = "https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip"

# Paths for downloaded files
video_rar_path = os.path.join("/content/UCF101", "UCF101.rar")
split_zip_path = os.path.join("/content/UCF101", "UCF101TrainTestSplits-RecognitionTask.zip")

# Download function with progress bar
def download_file(url, dest_path):
    print(f"Downloading {url} to {dest_path}")
    try:
        response = requests.get(url, stream=True, verify=False)  # Bypass SSL verification
        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            with open(dest_path, 'wb') as f, tqdm(
                desc=os.path.basename(dest_path),
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    bar.update(len(chunk))
            # Check file size (~6.9 GB for UCF101.rar)
            downloaded_size = os.path.getsize(dest_path)
            print(f"Downloaded file size: {downloaded_size / (1024**3):.2f} GB")
            return True
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return False

# Manual upload function
def manual_upload(dest_path):
    print(f"Please upload {os.path.basename(dest_path)} manually")
    uploaded = files.upload()
    if os.path.basename(dest_path) in uploaded:
        with open(dest_path, 'wb') as f:
            f.write(uploaded[os.path.basename(dest_path)])
        print(f"Uploaded {dest_path} successfully!")
        return True
    else:
        print(f"Manual upload failed for {dest_path}")
        return False

# Download files if they don't exist
if not os.path.exists(video_rar_path):
    if not download_file(video_url, video_rar_path):
        print("Download failed. Trying manual upload...")
        manual_upload(video_rar_path)
if not os.path.exists(split_zip_path):
    if not download_file(train_test_split_url, split_zip_path):
        print("Download failed. Trying manual upload...")
        manual_upload(split_zip_path)

print("Download completed!")

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.crcv.ucf.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
UCF101.rar: 100%|██████████| 6.46G/6.46G [02:20<00:00, 49.3MB/s]
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.crcv.ucf.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded file size: 6.46 GB


UCF101TrainTestSplits-RecognitionTask.zip: 100%|██████████| 111k/111k [00:00<00:00, 967kB/s] 

Downloaded file size: 0.00 GB
Download completed!


In [ ]:
import rarfile
import zipfile
import os

# Paths
video_rar_path = "/content/UCF101/UCF101.rar"
split_zip_path = "/content/UCF101/UCF101TrainTestSplits-RecognitionTask.zip"
video_dir = "/content/UCF101/videos"
annotation_dir = "/content/UCF101/annotations"

# Extract videos with error handling
print("Extracting videos...")
try:
    with rarfile.RarFile(video_rar_path) as rf:
        for file_info in rf.infolist():
            try:
                rf.extract(file_info, path=video_dir)
            except Exception as e:
                print(f"Failed to extract {file_info.filename}: {e}")
                continue
except Exception as e:
    print(f"Error opening RAR file: {e}")
    print("Please redownload UCF101.rar in Block 4 or upload manually.")

# Extract train/test splits
print("\nExtracting train/test splits...")
try:
    with zipfile.ZipFile(split_zip_path, 'r') as zf:
        zf.extractall(annotation_dir)
except Exception as e:
    print(f"Error extracting splits: {e}")

print("Extraction completed!")

Extracting videos...

Extracting train/test splits...
Extraction completed!


In [ ]:
!pip install RarFile

In [ ]:
import os

# YOLO dataset paths
yolo_data_dir = "/content/yolo_dataset"
train_img_dir = os.path.join(yolo_data_dir, "images", "train")
val_img_dir = os.path.join(yolo_data_dir, "images", "val")
train_label_dir = os.path.join(yolo_data_dir, "labels", "train")
val_label_dir = os.path.join(yolo_data_dir, "labels", "val")

# Create directories
for d in [train_img_dir, val_img_dir, train_label_dir, val_label_dir]:
    os.makedirs(d, exist_ok=True)

print("YOLO dataset structure created!")

YOLO dataset structure created!


In [ ]:
import os
import cv2
from ultralytics import YOLO
import numpy as np

# Selected action categories
action_categories = ["TrampolineJumping", "IceDancing", "Skiing", "Snowboarding", "WalkingWithDog"]
class_map = {cat: idx for idx, cat in enumerate(action_categories)}  # Map categories to class IDs

# Paths
video_dir = "/content/UCF101/videos"
annotation_dir = "/content/UCF101/annotations"
train_img_dir = "/content/yolo_dataset/images/train"
val_img_dir = "/content/yolo_dataset/images/val"
train_label_dir = "/content/yolo_dataset/labels/train"
val_label_dir = "/content/yolo_dataset/labels/val"

# Function to extract frames and annotate
def extract_frames_and_annotate(video_path, output_dir, label_dir, model, action_category, is_train=True):
    if not os.path.exists(video_path):
        print(f"Warning: Video not found: {video_path}")
        return
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    class_id = class_map[action_category]

    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % 10 == 0:  # Extract every 10th frame
            img_path = os.path.join(output_dir, f"{video_name}_frame{count}.jpg")
            cv2.imwrite(img_path, frame)

            # Use pre-trained YOLOv11n to detect humans
            results = model(frame)
            label_path = os.path.join(label_dir, f"{video_name}_frame{count}.txt")
            with open(label_path, 'w') as f:
                for det in results[0].boxes:
                    if det.cls == 0:  # Class 0 is 'person' in COCO
                        x, y, w, h = det.xywh[0].cpu().numpy()
                        img_h, img_w = frame.shape[:2]
                        # Normalize coordinates for YOLO format
                        x_center = x / img_w
                        y_center = y / img_h
                        width = w / img_w
                        height = h / img_h
                        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
        count += 1
    cap.release()

# Load pre-trained YOLOv11n for pseudo-annotations
model = YOLO("/content/yolo11n.pt")

# Read train and validation video lists
split_dir = os.path.join(annotation_dir, "/content/UCF101/UCF101TrainTestSplits-RecognitionTask.zip")
train_list_path = os.path.join(split_dir, "/content/UCF101/annotations/ucfTrainTestlist/trainlist01.txt")
val_list_path = os.path.join(split_dir, "/content/UCF101/annotations/ucfTrainTestlist/testlist01.txt")

train_videos = []
val_videos = []
try:
    with open(train_list_path, 'r') as train_file:
        for line in train_file:
            video = line.strip().split()[0]
            for cat in action_categories:
                if cat in video:
                    train_videos.append((video, cat))
    with open(val_list_path, 'r') as val_file:
        for line in val_file:
            video = line.strip()
            for cat in action_categories:
                if cat in video:
                    val_videos.append((video, cat))
except FileNotFoundError as e:
    print(f"Error: Split file not found: {e}")
    print("Ensure Block 5 extracted splits correctly.")

# Extract frames and annotations
for video, category in train_videos:
    video_path = os.path.join(video_dir, "UCF-101", video)
    extract_frames_and_annotate(video_path, train_img_dir, train_label_dir, model, category, is_train=True)
for video, category in val_videos:
    video_path = os.path.join(video_dir, "UCF-101", video)
    extract_frames_and_annotate(video_path, val_img_dir, val_label_dir, model, category, is_train=False)

print("Frame extraction and annotation completed!")

Streaming output truncated to the last 5000 lines.
Speed: 2.6ms preprocess, 17.3ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 4 cars, 16.6ms
Speed: 3.5ms preprocess, 16.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 car, 18.8ms
Speed: 2.8ms preprocess, 18.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 18.0ms
Speed: 2.7ms preprocess, 18.0ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 bus, 13.0ms
Speed: 2.3ms preprocess, 13.0ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 car, 1 bus, 19.0ms
Speed: 2.8ms preprocess, 19.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 15.3ms
Speed: 2.8ms preprocess, 15.3ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 dog, 19.0ms
Speed: 2.8ms preproces

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import yaml
import os

# YOLO dataset paths
yolo_data_dir = "/content/yolo_dataset"
train_img_dir = os.path.join(yolo_data_dir, "images", "train")
val_img_dir = os.path.join(yolo_data_dir, "images", "val")

# Action categories
action_categories = ["TrampolineJumping", "IceDancing", "Skiing", "Snowboarding", "WalkingWithDog"]

# Create YAML file
data_yaml = {
    "train": train_img_dir,
    "val": val_img_dir,
    "nc": len(action_categories),
    "names": action_categories
}
with open(os.path.join(yolo_data_dir, "data.yaml"), 'w') as f:
    yaml.dump(data_yaml, f)

print("YOLO data.yaml file created!")

YOLO data.yaml file created!


In [ ]:
from ultralytics import YOLO

# Load pre-trained model
model = YOLO("/content/yolo11n.pt")

# Train the model
print("Starting YOLOv11n training...")
model.train(
    data="/content/yolo_dataset/data.yaml",
    epochs=10,
    imgsz=640,
    batch=16,
    device=0,  # Use GPU
    project="/content/drive/MyDrive/yolo_training",
    name="yolov11n_ucf101_multi",
    patience=5
)

print("Training completed!")d

Starting YOLOv11n training...
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11n_ucf101_multi, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=

100%|██████████| 755k/755k [00:00<00:00, 17.7MB/s]

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,815 parameters, 2,590,799 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 666.6±348.2 MB/s, size: 19.1 KB)


train: Scanning /content/yolo_dataset/labels/train... 8614 images, 1496 backgrounds, 0 corrupt: 100%|██████████| 8614/8614 [00:03<00:00, 2494.26it/s]


...

      10/10      2.64G     0.7299     0.7226      1.015         10        640: 100%|██████████| 539/539 [02:33<00:00,  3.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:26<00:00,  4.30it/s]

                   all       3601       4902      0.761      0.728      0.793      0.624



10 epochs completed in 0.506 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_training/yolov11n_ucf101_multi/weights/last.pt, 5.5MB
Optimizer stripped from /content/drive/MyDrive/yolo_training/yolov11n_ucf101_multi/weights/best.pt, 5.5MB

Validating /content/drive/MyDrive/yolo_training/yolov11n_ucf101_multi/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:28<00:00,  3.91it/s]


                   all       3601       4902      0.761      0.727      0.793      0.624
     TrampolineJumping        501        805      0.809      0.698      0.831      0.634
            IceDancing       1145       2282      0.806      0.832      0.882      0.699
                Skiing        755        962      0.694      0.701      0.723      0.572
        WalkingWithDog        664        853      0.734      0.679      0.737      0.591
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_training/yolov11n_ucf101_multi
Training completed!


In [14]:
import shutil
from ultralytics import YOLO

# Load trained model
model = YOLO("/content/drive/MyDrive/yolo_training/yolov11n_ucf101_multi/weights/best.pt")

# Evaluate
results = model.val()
print("Validation results:", results)

# Export to ONNX
model.export(format="onnx")
print("Model exported to ONNX format.")

# Save to Google Drive
shutil.copytree("/content/drive/MyDrive/yolo_training/yolov11n_ucf101_multi",
                "/content/drive/MyDrive/yolo_training_final")
print("Model saved to Google Drive.")

Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 662.3±303.1 MB/s, size: 19.0 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 3601 images, 536 backgrounds, 0 corrupt: 100%|██████████| 3601/3601 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 226/226 [00:34<00:00,  6.60it/s]


                   all       3601       4902       0.76      0.728      0.793      0.624
     TrampolineJumping        501        805      0.808        0.7      0.831      0.634
            IceDancing       1145       2282      0.806      0.833      0.882        0.7
                Skiing        755        962      0.693      0.701      0.723      0.573
        WalkingWithDog        664        853      0.733      0.679      0.736      0.591
Speed: 0.6ms preprocess, 3.0ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val
Validation results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79b30e2b5190>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.00400

In [20]:
!python /content/validate.py --weights /content/drive/MyDrive/yolo_training/yolov11n_ucf101_multi/weights/best.pt --test_videos /content/k.mp4 --output /content/validation_results


Processing video: /content/k.mp4

0: 320x640 4 IceDancings, 96.1ms
Speed: 5.0ms preprocess, 96.1ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 IceDancings, 75.2ms
Speed: 2.5ms preprocess, 75.2ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 IceDancings, 70.1ms
Speed: 3.0ms preprocess, 70.1ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 IceDancings, 68.6ms
Speed: 2.8ms preprocess, 68.6ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 IceDancings, 79.6ms
Speed: 2.6ms preprocess, 79.6ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 IceDancings, 66.9ms
Speed: 3.0ms preprocess, 66.9ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 IceDancings, 73.5ms
Speed: 2.9ms preprocess, 73.5ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 IceDancings, 67.9ms
Speed: 3.0ms pr